In [1]:
import pandas as pd

df = pd.read_excel('sample200_emotions_ner.xlsx')
df

,Unnamed: 0,tweetURL,tweetID,date,tweetContent,userLocation,isUserVerified,replyCount,retweetCount,likeCount,quoteCount,tweetLanguage,sourceLabel,coordinates,place,emotion,score,people,locations,organizations
0,14041,https://twitter.com/priyadarshy/status/1454029...,1454029742882033920,2021-10-29T10:17:59+00:00,Thank #ATCE2021 for inviting me to speak. Acce...,US,False,0,1,0,0,en,LinkedIn,NaN,NaN,joy,0.664280,NaN,NaN,NaN
1,3680,https://twitter.com/greenreccompany/status/145...,1458963009406181120,2021-11-12T01:01:02+00:00,We're working with an international #NetZero c...,London,False,0,0,0,0,en,Hootsuite Inc.,NaN,NaN,fear,0.924377,I ##eu ##an Jones i ##an ##ale,Singapore Asia,NaN
2,257,https://twitter.com/SEEBeyondCoal/status/14609...,1460994665335995904,2021-11-17T15:34:06+00:00,Diversify energy sources\n#EnergyTransition #...,Southeast Europe,False,0,0,0,0,en,Twitter Web App,NaN,NaN,fear,0.783113,NaN,NaN,Energy
3,16525,https://twitter.com/GlobalGoalsEU/status/14529...,1452916785825690112,2021-10-26T08:35:30+00:00,🔵#EnergyCommunities are essential for the #Ene...,Worldwide,False,0,0,3,0,en,Twitter for iPhone,NaN,NaN,fear,0.942047,NaN,NaN,NaN
4,6147,https://twitter.com/IGEFSO/status/145793997231...,1457939972317748992,2021-11-09T05:15:51+00:00,.@ntpclimited order Inox #Wind turbines for 15...,"New Delhi, India",False,0,0,0,0,en,Buffer,NaN,NaN,fear,0.902760,NaN,India,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,15981,https://twitter.com/JWNenergy/status/145309324...,1453093244036780032,2021-10-26T20:16:41+00:00,Join us for a candid conversation about the st...,"Calgary, Alberta, Canada",False,1,3,2,0,en,Twitter Web App,NaN,NaN,neutral,0.913836,Peter Te ##rt ##zaki ##an Ross Manning,Canada,Canadian Energy Industry IBM Consul ##ting
196,2607,https://twitter.com/hiharveen/status/145982196...,1459821963917118976,2021-11-14T09:54:13+00:00,Fundamental threat to energy transition. \n\nS...,NaN,False,0,0,0,0,en,Twitter Web App,NaN,NaN,fear,0.976220,NaN,Paris,NaN
197,3963,https://twitter.com/CavagnaGroup/status/145881...,1458813016737210112,2021-11-11T15:05:01+00:00,The @LiquidGasUK Conference has been a great c...,Brescia Italy,False,0,1,8,1,en,Twitter Web App,NaN,NaN,fear,0.511848,NaN,NaN,Force @ C ##ava ##gna ##U ##K Group
198,1547,https://twitter.com/Indian10000000/status/1460...,1460457896970383104,2021-11-16T04:01:11+00:00,On the technology hurdles in energy transitio...,NaN,False,0,3,7,0,en,Twitter for Android,NaN,NaN,fear,0.622184,NaN,NaN,RS


In [13]:
# test ollama 
import ollama
ollama.chat(model='qwen2.5:3b', 
            messages=[{'role': 'user', 'content': "hello"}])

ChatResponse(model='qwen2.5:3b', created_at='2025-04-03T09:13:26.3268648Z', done=True, done_reason='stop', total_duration=22651043500, load_duration=21039965100, prompt_eval_count=30, prompt_eval_duration=600294700, eval_count=53, eval_duration=1003682700, message=Message(role='assistant', content="Hello! How can I assist you today? Feel free to ask any questions or let me know if there's anything specific you'd like to explore or discuss. Whether it's about technology, information retrieval, or any other topic, I'm here to help.", images=None, tool_calls=None))

# use ollama with qwen2.5:3b to normlize country name for userLocation column

In [14]:
import pandas as pd

# Function to normalize country name using Ollama
def get_normalized_country(location):
    if pd.isna(location):
        return None
    
    prompt = f"""Given this location text: "{location}", return only the standard country name.
    If you can't determine the country, return None.
    Only return the country name, nothing else."""
    
    try:
        response = ollama.chat(model='qwen2.5:3b', 
                             messages=[{'role': 'user', 'content': prompt}])
        country = response['message']['content'].strip()
        return None if country.lower() in ['none', 'unknown', 'n/a'] else country
    except:
        return None



In [15]:
# Test the function with some sample locations
sample_locations = [
    "London",
    "New Delhi, India",
    "Southeast Europe",
    "Calgary, Alberta, Canada",
    "Brescia Italy",
    None
]

print("Testing country normalization:")
for location in sample_locations:
    result = get_normalized_country(location)
    location_str = str(location) if location is not None else "None"
    print(f"Input: {location_str:<25} -> Output: {result}")

Testing country normalization:
Input: London                    -> Output: United Kingdom
Input: New Delhi, India          -> Output: India
Input: Southeast Europe          -> Output: None
Input: Calgary, Alberta, Canada  -> Output: Canada
Input: Brescia Italy             -> Output: Italy
Input: None                      -> Output: None


In [16]:
# Create a new column with normalized country names
from tqdm.notebook import tqdm
tqdm.pandas()
df['normalized_country'] = df['userLocation'].progress_apply(get_normalized_country)


  0%|          | 0/200 [00:00<?, ?it/s]

In [17]:
df[['userLocation', 'normalized_country']]

,userLocation,normalized_country
0,US,United States
1,London,United Kingdom
2,Southeast Europe,None
3,Worldwide,None
4,"New Delhi, India",India
...,...,...
195,"Calgary, Alberta, Canada",Canada
196,NaN,None
197,Brescia Italy,Italy
198,NaN,None


In [19]:
df.to_excel('sample200_emotions_ner.xlsx', index=False)